In [1]:
# importar de drive

from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
!ls /content/drive/MyDrive/Preparatorio/Optimizacion/datos

df_parcial.csv		      mantenimiento_equipos_reducido_con_pooh.csv
df_parcial.gsheet	      mantenimiento_equipos_reducido.csv
mantenimiento_equipos.csv     mantenimiento_equipos_reducido_sin_pooh.csv
mantenimiento_equipos.gsheet  mantenimiento_equipos_running.csv


In [ ]:
# base folder en drive
BASE_FOLDER = '/content/drive/MyDrive/Preparatorio/Optimizacion/datos'

In [ ]:
#importar csv de drive
import pandas as pd
df = pd.read_csv(f'{BASE_FOLDER}/mantenimiento_equipos.csv')

<ipython-input-171-24afae4b3606>:3: DtypeWarning: Columns (28,30,31,32,33,35,40,48,49,54,57,70,72,73,85,86,87,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{BASE_FOLDER}/mantenimiento_equipos.csv')


In [ ]:
#importar csv de drive
import pandas as pd
df_reducido = pd.read_csv(f'{BASE_FOLDER}/df_parcial.csv')

In [ ]:
df_reducido[['STOP_INDEX', 'STOP_GROUP']]

,STOP_INDEX,STOP_GROUP
0,16.0,ELECTRIC
1,16.0,ELECTRIC
2,48.0,WELL
3,48.0,WELL
4,48.0,WELL
...,...,...
8284,11.0,ELECTIVE
8285,32.0,MECHANICAL
8286,39.0,UNKNOWN
8287,14.0,ELECTRIC


In [ ]:
# hacer groupby por stop index y haga un count solo por la columna de stop group
df_reducido[df_reducido['STOP_INDEX'] == 2][['STOP_GROUP', 'REASON_FOR']].value_counts()

,count
STOP_GROUP,
ELECTIVE,108


In [ ]:
df.shape[0]

8289

In [ ]:
df.columns = df.columns.str.upper()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')

In [ ]:
# eliminar las primeras 14 columnas de df
df = df.iloc[:, 23:]

In [ ]:
df.shape[0]

8289

In [ ]:
# drop columnas 'Unnamed: 0.1', 'Unnamed: 0'
df_reducido = df_reducido.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [ ]:
df_reducido.columns

Index(['COUNTRY', 'ASSET', 'COMPLETION_NUMBER', 'ACTIVATION_TYPE', 'SUPPLIER',
       'ENVIRONMENT', 'RUNNING_STATUS', 'INSTALLATION_DATE', 'FAILURE_DATE',
       'ACLS_DATE', 'POOH_DATE', 'LAST_UPDATE', 'RUN_LIFE', 'RUN_LIFE_2',
       'START_DATE', 'START_UP_DELAY_2', 'START_UP_DELAY', 'ORDINAL_CHECK',
       'MAX_INSTALLS', 'STOP_INDEX', 'STOP_GROUP', 'LAST_INSTALL'],
      dtype='object')

In [ ]:
df_concat = pd.concat([df_reducido, df], axis=1)

In [ ]:
df_concat.shape[0]

8289

In [ ]:
df_concat['START_DATE'] = pd.to_datetime(df_concat['START_DATE'])
df_concat['INSTALLATION_DATE'] = pd.to_datetime(df_concat['INSTALLATION_DATE'])
df_concat['FAILURE_DATE'] = pd.to_datetime(df_concat['FAILURE_DATE'])
df_concat['ACLS_DATE'] = pd.to_datetime(df_concat['ACLS_DATE'])
df_concat['POOH_DATE'] = pd.to_datetime(df_concat['POOH_DATE'])

In [ ]:
df_concat.iloc[101][:18]

,101
COUNTRY,ARGENTINA
ASSET,Y8PMI3
COMPLETION_NUMBER,18
ACTIVATION_TYPE,ESP
SUPPLIER,MAC
ENVIRONMENT,OFFSHORE
RUNNING_STATUS,PULLED
INSTALLATION_DATE,2022-10-01 00:00:00
FAILURE_DATE,2023-07-18 00:00:00
ACLS_DATE,NaT


In [ ]:
# aquellos valores NaT de df_concat['START_DATE'] completalos de INTALLATION_DATE
df_concat.loc[df_concat['START_DATE'].isna(),
              'START_DATE'] = df_concat.loc[df_concat['START_DATE'].isna(),
                                            'INSTALLATION_DATE']

In [ ]:
df_concat.iloc[101][:18]

,101
COUNTRY,ARGENTINA
ASSET,Y8PMI3
COMPLETION_NUMBER,18
ACTIVATION_TYPE,ESP
SUPPLIER,MAC
ENVIRONMENT,OFFSHORE
RUNNING_STATUS,PULLED
INSTALLATION_DATE,2022-10-01 00:00:00
FAILURE_DATE,2023-07-18 00:00:00
ACLS_DATE,NaT


In [ ]:
df_concat['RUN_LIFE_2'] = pd.to_timedelta(df_concat['RUN_LIFE_2'])

In [ ]:
# exportar csv
df_reducido.to_csv(f'{BASE_FOLDER}/df_parcial.csv')

El registro 3283 es un dato sin calidad.

In [ ]:
# llenar con 0 días
df_reducido['START_UP_DELAY_2'] = df_reducido['START_UP_DELAY_2'].fillna(pd.Timedelta(days=0))

In [ ]:
# comparar START_UP_DELAY y START_UP_DELAY_2 para ver si son iguales
(df_reducido['START_UP_DELAY'] == df_reducido['START_UP_DELAY_2'].dt.days).value_counts()

,count
True,8289


In [ ]:
df_reducido['ORDINAL_CHECK'] = df_concat['ORDINAL_CHECK'].str.upper().str.replace(' ', '_')

In [ ]:
df_reducido['ORDINAL_CHECK'].value_counts()

,count
ORDINAL_CHECK,
NORMAL,6896
NEW_WELL,1381
REDUNDANT_WELLS,10
DATE_ERROR,1
MISSING_WELLS,1


In [ ]:
df_reducido['MAX_INSTALLS'] = df_concat['MAX_INSTALLS']

In [ ]:
df_reducido['STOP_INDEX'] = df_concat['STOP_INDEX']

In [ ]:
df_concat['STOP_GROUP'] = df_concat['STOP_GROUP'].str.upper().str.replace(' ', '_')

In [ ]:
df_reducido['STOP_GROUP'] = df_concat['STOP_GROUP']

In [ ]:
df_concat['REASON_FOR_STOP_OR_POOH'].unique()

array(['Electric Failure - Unspecified', 'Low Productivity', 'Unknown',
       'Cable Failure', 'Installation Mistake',
       'Pump Stuck, including scale', 'Intake, pump, or tubing plugged',
       'Tubing or comp acc failure', 'ESP Downsize', 'Operation',
       'Broken Shaft or Coupling', 'High BSW', '-', 'Low efficiency',
       'Corrosion', 'Pump Failure', 'Wrong ESP or Change ESP design',
       'MLE or Pothead failure', 'Splice Failure',
       'Temporarily Abandoned', 'Mechanical Failure - unspecified',
       'Unspecified Failure', 'ESP Upsize', 'Motor failure',
       'WH Penetrator or Connector', 'Mechanical - Overload',
       'Protector or seal failure', nan, 'Motor Failure',
       'Elective - Unspecified',
       'Packer penetrator/connector or HAS Failure',
       'Acidize/Stimulation', 'Electric Failure - Multiple Elements',
       'Permanently Abandoned', 'Water Shutoff',
       'Zone change or addition', 'New perforations or re-perforate',
       'Converted to Injec

In [ ]:
df_concat['STATUS_CHECK'].value_counts()

In [ ]:
df_concat[df['STATUS_CHECK']=='Data Error']

,COUNTRY,ASSET,COMPLETION_NUMBER,ACTIVATION_TYPE,SUPPLIER,ENVIRONMENT,RUNNING_STATUS,INSTALLATION_DATE,FAILURE_DATE,ACLS_DATE,...,H2S_(Y/N),CHEMICAL_INJECTION_(Y/N),USAGE_MOTOR,USAGE_PUMP,USAGE_PROTECTOR,USAGE_INTAKE,USAGE_ROUND,USAGE_FLAT,USAGE_GAUGE,PUMP_IS_NEW?
3283,INDIA,LKLMIMX2GI,3,ESP,GOONER,OFFSHORE,PULLED,NaT,NaT,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown


In [ ]:
df_concat['DATE/TIME_CHECK'].value_counts()

,count
DATE/TIME_CHECK,
OK,8289


In [ ]:
df_reducido['ORDINAL_CHECK'].value_counts()

,count
ORDINAL_CHECK,
NORMAL,6896
NEW_WELL,1381
REDUNDANT_WELLS,10
DATE_ERROR,1
MISSING_WELLS,1


In [ ]:
df_concat['LAST_INSTALL'].value_counts()

,count
LAST_INSTALL,
No,6859
Yes,1430


In [ ]:
# binarizar yes 1 no 0
df_concat['LAST_INSTALL'] = df_concat['LAST_INSTALL'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
df_reducido['LAST_INSTALL'] = df_concat['LAST_INSTALL']

In [ ]:
df_reducido['LAST_INSTALL'].value_counts()

,count
LAST_INSTALL,
0,6859
1,1430


In [ ]:
df_reducido[df_reducido['LAST_INSTALL']==1][df_reducido['RUNNING_STATUS']=='RUNNING'].shape[0]

<ipython-input-62-720cf645d4ed>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_reducido[df_reducido['LAST_INSTALL']==1][df_reducido['RUNNING_STATUS']=='RUNNING'].shape[0]


575

De los 1430 equipos que están etiquetados como ultima instalción, hay 575 en operación. La diferencia indica posos que ya no cuentan con equipos.

In [ ]:
df_concat['CHECKS'].value_counts()

,count
CHECKS,
OK,8289


In [ ]:
df_concat[df_concat['STOP_INDEX']==1].shape[0]

597

In [ ]:
df_concat['RUNNING_STATUS'].value_counts()

,count
RUNNING_STATUS,
PULLED,6922
RUNNING,575
ACLS,542
FAILED,174
IDLE,76


In [ ]:
df_concat[df_concat['STOP_INDEX']==1]['RUNNING_STATUS'].value_counts()

,count
RUNNING_STATUS,
RUNNING,575
IDLE,37
PULLED,12
ACLS,4
FAILED,3


In [ ]:
indices_nan = df_concat[df_concat['RUNNING_STATUS']=='RUNNING'][df_concat['STOP_INDEX'] != 1][['COMPLETION_NUMBER', 'STOP_INDEX', 'STOP_GROUP','REASON_FOR_STOP_OR_POOH']].index

<ipython-input-98-6b30af861ba1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indices_nan = df_concat[df_concat['RUNNING_STATUS']=='RUNNING'][df_concat['STOP_INDEX'] != 1][['COMPLETION_NUMBER', 'STOP_INDEX', 'STOP_GROUP','REASON_FOR_STOP_OR_POOH']].index


In [ ]:
df_concat.loc[indices_nan, 'STOP_INDEX'] = 1

In [ ]:
df_concat.iloc[1831][:30]

,1831
COUNTRY,ARGENTINA
ASSET,VMLM6X
COMPLETION_NUMBER,12
ACTIVATION_TYPE,ESP
SUPPLIER,GOONER
ENVIRONMENT,OFFSHORE
RUNNING_STATUS,RUNNING
INSTALLATION_DATE,2023-03-26 00:00:00
FAILURE_DATE,NaT
ACLS_DATE,NaT


In [ ]:
import numpy as np
df_concat.at[1831, 'STOP_INDEX'] = 1
df_concat.at[1831, 'STOP_GROUP'] = np.nan
df_concat.at[1831, 'REASON_FOR_STOP_OR_POOH'] = np.nan

In [ ]:
df_concat.at[7334, 'STOP_INDEX'] = 1
df_concat.at[7334, 'STOP_GROUP'] = np.nan
df_concat.at[7334, 'REASON_FOR_STOP_OR_POOH'] = np.nan

In [ ]:
df_concat.at[7532, 'STOP_INDEX'] = 1
df_concat.at[7532, 'STOP_GROUP'] = np.nan
df_concat.at[7532, 'REASON_FOR_STOP_OR_POOH'] = np.nan

In [ ]:
df_reducido['STOP_INDEX'].iloc[1831]

1.0

In [ ]:
# importar csv
df_partial = pd.read_csv(f'{BASE_FOLDER}/df_parcial.csv')

In [ ]:
df_partial['STOP_INDEX'] = df_concat['STOP_INDEX']

In [ ]:
df_partial['STOP_GROUP'] = df_concat['STOP_GROUP']

In [ ]:
# guardar csv
df_partial.to_csv(f'{BASE_FOLDER}/df_parcial.csv')

In [ ]:
# convertir a mayuscula
df_concat['CLASS'] = df_concat['CLASS'].str.upper()

In [ ]:
df_concat['CLASS'].value_counts()

,count
CLASS,
F,4825
E,933
N,731
U,444
S,174
C,172
L,168
A,65
N/A,1


In [ ]:
df_concat[~pd.isna(df_concat['FAILURE_DATE'])]['CLASS'].value_counts()

,count
CLASS,
F,4217
E,659
N,520
U,395
C,66
S,52
A,25
L,5


In [ ]:
df_concat['RUNNING_STATUS'].value_counts()

,count
RUNNING_STATUS,
PULLED,6922
RUNNING,575
ACLS,542
FAILED,174
IDLE,76


## CLASS

In [ ]:
# .upper a class
df_concat['CLASS'] = df_concat['CLASS'].str.upper()

In [ ]:
df_concat['CLASS'].value_counts(dropna = False)

,count
CLASS,
F,4825
E,933
NaN,776
N,731
U,444
S,174
C,172
L,168
A,65


In [ ]:
df_concat\
 [df_concat['RUNNING_STATUS']=='RUNNING']\
  ['CLASS'].value_counts(dropna=False)

,count
CLASS,
NaN,571
F,2
S,1
N,1


In [ ]:
df_concat\
 [df_concat['RUNNING_STATUS']=='RUNNING']\
  ['CLASS'].value_counts(dropna=False)

,count
CLASS,
NaN,571
N/A,2


Se revisa el registro KZWM2X-4 y se verifica que efectivamente está fuera de servicio y no running. Se agrega como fecha de falla el 19-10-2023. Además el registro K7UMI3-2, se asume que la fecha de falla es 22-04-2024.

El registro VMLM6X-12 está running y por tanto su class debe ser N/A.

In [ ]:
df_concat\
 [(df_concat['RUNNING_STATUS']=='RUNNING') &
  (df_concat['CLASS'].isin(['F', 'S', 'N']))].iloc[:, 1:26]

,ASSET,COMPLETION_NUMBER,ACTIVATION_TYPE,SUPPLIER,ENVIRONMENT,RUNNING_STATUS,INSTALLATION_DATE,FAILURE_DATE,ACLS_DATE,POOH_DATE,...,START_UP_DELAY,ORDINAL_CHECK,MAX_INSTALLS,STOP_INDEX,STOP_GROUP,LAST_INSTALL,REASON_FOR_STOP_OR_POOH,CLASS,ESP_FAIL?,TUBING_SIZE_(INCH)
1494,VMLM28,6,ESP,GOONER,OFFSHORE,RUNNING,2023-07-25,NaT,NaT,NaT,...,6,NORMAL,6,1.0,-,1,-,S,No,"3,5"


In [ ]:
df_concat.iloc[[1494, 1831, 7532, 7334]].iloc[:, 1:26]

,ASSET,COMPLETION_NUMBER,ACTIVATION_TYPE,SUPPLIER,ENVIRONMENT,RUNNING_STATUS,INSTALLATION_DATE,FAILURE_DATE,ACLS_DATE,POOH_DATE,...,START_UP_DELAY,ORDINAL_CHECK,MAX_INSTALLS,STOP_INDEX,STOP_GROUP,LAST_INSTALL,REASON_FOR_STOP_OR_POOH,CLASS,ESP_FAIL?,TUBING_SIZE_(INCH)
1494,VMLM28,6,ESP,GOONER,OFFSHORE,RUNNING,2023-07-25,NaT,NaT,NaT,...,6,NORMAL,6,1.0,-,1,-,N/A,No,"3,5"
1831,VMLM6X,12,ESP,GOONER,OFFSHORE,RUNNING,2023-03-26,NaT,NaT,NaT,...,2,NORMAL,12,1.0,-,1,-,N/A,No,"3,5"
7532,K7UMI3,2,ESP,QUAKER,ONSHORE,PULLED,2023-12-04,2024-04-22,NaT,NaT,...,0,NORMAL,2,39.0,UNKNOWN,1,UNKNOWN,F,Yes,"2,875"
7334,KZWM2X,4,ESP,QUAKER,ONSHORE,PULLED,2019-10-19,2023-10-19,NaT,NaT,...,1,NORMAL,4,40.0,UNSPECIFIED_FAILURE,1,UNSPECIFIED FAILURE,F,Yes,"3,5"


In [ ]:
df_concat.loc[1831, 'REASON_FOR_STOP_OR_POOH'] = '-'

In [ ]:
df_concat.loc[1831, 'STOP_GROUP'] = '-'

In [ ]:
df_concat.loc[1494, 'CLASS'] = 'N/A'

In [ ]:
df_concat.loc[1831, 'CLASS'] = 'N/A'

In [ ]:
df_concat.loc[7532, 'STOP_INDEX'] = 39

In [ ]:
df_concat.loc[7532, 'STOP_GROUP'] = 'UNKNOWN'

In [ ]:
df_concat.loc[7532, 'RUNNING_STATUS'] = 'PULLED'

In [ ]:
df_concat.loc[7532, 'RUN_LIFE'] = df_concat.loc[7532, 'RUN_LIFE'].days

In [ ]:
df_concat.loc[7532, 'RUN_LIFE'] = \
df_concat.loc[7532, 'FAILURE_DATE'] - df_concat.loc[7532, 'INSTALLATION_DATE']

In [ ]:
df_concat.loc[7532, 'FAILURE_DATE'] = pd.to_datetime('2024-04-22')

In [ ]:
df_concat.loc[7334, 'RUN_LIFE'] = df_concat.loc[7334, 'RUN_LIFE'].days

In [ ]:
df_concat.loc[7334, 'RUN_LIFE_2']

Timedelta('1461 days 00:00:00')

In [ ]:
# hacer la diferencia en dias entre las fechas 19-10-2019 y 9-01-2025
pd.to_datetime('2025-01-09') - pd.to_datetime('2019-10-19')

Timedelta('1909 days 00:00:00')

In [ ]:
df_concat.loc[7334, 'FAILURE_DATE'] - df_concat.loc[7334, 'INSTALLATION_DATE']

Timedelta('1461 days 00:00:00')

In [ ]:
df_concat.loc[7334, 'RUN_LIFE_2'] = \
df_concat.loc[7334, 'FAILURE_DATE'] - df_concat.loc[7334, 'INSTALLATION_DATE']

In [ ]:
df_concat.loc[7334, 'STOP_GROUP'] = 'UNSPECIFIED_FAILURE'

In [ ]:
df_concat.loc[7334, 'RUNNING_STATUS'] = 'PULLED'

In [ ]:
df_concat.loc[7334].iloc[:27]

,7334
COUNTRY,TURKEY
ASSET,KZWM2X
COMPLETION_NUMBER,4
ACTIVATION_TYPE,ESP
SUPPLIER,QUAKER
ENVIRONMENT,ONSHORE
RUNNING_STATUS,PULLED
INSTALLATION_DATE,2019-10-19 00:00:00
FAILURE_DATE,2023-10-19 00:00:00
ACLS_DATE,NaT


In [ ]:
df_concat[df_concat['ASSET'] == 'KZWM2X'].iloc[:,: 27]

,COUNTRY,ASSET,COMPLETION_NUMBER,ACTIVATION_TYPE,SUPPLIER,ENVIRONMENT,RUNNING_STATUS,INSTALLATION_DATE,FAILURE_DATE,ACLS_DATE,...,ORDINAL_CHECK,MAX_INSTALLS,STOP_INDEX,STOP_GROUP,LAST_INSTALL,REASON_FOR_STOP_OR_POOH,CLASS,ESP_FAIL?,TUBING_SIZE_(INCH),TUBING_WEIGHT_(PPF)
7331,TURKEY,KZWM2X,1,ESP,QUAKER,ONSHORE,PULLED,1999-09-15,NaT,NaT,...,NEW_WELL,4,42.0,WELL,0,CONVERTED LIFT METHOD,C,No,NaN,NaN
7332,TURKEY,KZWM2X,2,ESP,MIXED,ONSHORE,PULLED,2012-05-21,2013-01-04,NaT,...,NORMAL,4,8.0,ELECTIVE,0,ESP UPSIZE,E,No,NaN,NaN
7333,TURKEY,KZWM2X,3,ESP,QUAKER,ONSHORE,PULLED,2013-12-26,2019-05-19,NaT,...,NORMAL,4,14.0,ELECTRIC,0,CABLE FAILURE,F,Yes,NaN,NaN
7334,TURKEY,KZWM2X,4,ESP,QUAKER,ONSHORE,RUNNING,2019-10-19,NaT,NaT,...,NORMAL,4,1.0,NaN,1,UNSPECIFIED FAILURE,F,Yes,"3,5","9,3"


In [ ]:
df_concat.iloc[7334]

,7334
COUNTRY,TURKEY
ASSET,KZWM2X
COMPLETION_NUMBER,4
ACTIVATION_TYPE,ESP
SUPPLIER,QUAKER
...,...
USAGE_INTAKE,1
USAGE_ROUND,0
USAGE_FLAT,NaN
USAGE_GAUGE,0


In [ ]:
df_concat\
 [df_concat['RUNNING_STATUS']=='ACLS']\
  [df_concat['CLASS'] == 'C']['REASON_FOR_STOP_OR_POOH'].value_counts()

<ipython-input-198-776920a3e026>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_concat\


,count
REASON_FOR_STOP_OR_POOH,
Converted Lift Method,99
Converted to Injector,23
Electric Failure - Unspecified,3
Broken Shaft or Coupling,2
Motor Failure,1
"Pump Stuck, including scale",1
High BSW,1
Pump Failure,1


In [ ]:
df_concat.loc[
    (df_concat['RUNNING_STATUS'] == 'ACLS') & (df_concat['CLASS'] == 'F'),
    'RUNNING_STATUS'
] = 'PULLED'

In [ ]:
df_concat\
 [df_concat['RUNNING_STATUS']=='ACLS']\
  [df_concat['CLASS'] == 'F']['RUNNING_STATUS'].shape[0]

<ipython-input-189-df82df0ab6bc>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_concat\


53

In [ ]:
df_concat\
 [df_concat['RUNNING_STATUS']=='ACLS']\
  [df_concat['CLASS'] == 'F'][df_concat['REASON_FOR_STOP_OR_POOH'] == 'Unknown']\
   [['REASON_FOR_STOP_OR_POOH', 'ESP_FAIL?']]

<ipython-input-175-8b209215241b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_concat\
<ipython-input-175-8b209215241b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_concat\


,REASON_FOR_STOP_OR_POOH,ESP_FAIL?
195,Unknown,Yes
823,Unknown,Yes
824,Unknown,Yes
830,Unknown,Yes
1721,Unknown,Yes
4280,Unknown,Yes
6617,Unknown,Yes
6624,Unknown,Yes
7194,Unknown,Yes
8245,Unknown,Yes


In [ ]:
# ACLS
# F
# reason_for_stop_or_pooh

In [ ]:
indices = df_concat[
    (df_concat['RUNNING_STATUS'] == 'ACLS') &
    (df_concat['CLASS'].isin(['F', 'N', 'E']))
][['CLASS', 'RUNNING_STATUS', 'REASON_FOR_STOP_OR_POOH']]['REASON_FOR_STOP_OR_POOH'].value_counts().index


In [ ]:
indices

Index(['Electric Failure - Unspecified', 'Unknown',
       'Pump Stuck, including scale', 'Tubing or comp acc failure',
       'Converted to Injector', 'Broken Shaft or Coupling', 'High BSW',
       'Elective - Unspecified', 'Converted Lift Method',
       'Unspecified Failure', 'Corrosion', 'Cable Failure', 'Low Productivity',
       'Low efficiency', 'Abandoned', 'Operation', 'Zone change or addition',
       'Motor Failure', 'Temporarily Abandoned'],
      dtype='object', name='REASON_FOR_STOP_OR_POOH')

In [ ]:
df_concat['REASON_FOR_STOP_OR_POOH'].value_counts()

,count
REASON_FOR_STOP_OR_POOH,
Unknown,1396
Electric Failure - Unspecified,1342
Cable Failure,805
Motor Failure,804
-,596
...,...
Paraffin,1
gas lock,1
ESP Deepening,1


In [ ]:
df_concat[df_concat['RUNNING_STATUS']=='ACLS'][df_concat['CLASS']=='U']['ESP_FAIL?'].value_counts()

<ipython-input-178-f268cab8a724>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_concat[df_concat['RUNNING_STATUS']=='ACLS'][df_concat['CLASS']=='U']['ESP_FAIL?'].value_counts()


,count
ESP_FAIL?,
Unknown,9
No,3
Yes,1


In [ ]:
df_concat[df_concat['RUNNING_STATUS']=='PULLED'][df_concat['CLASS']=='U']['ESP_FAIL?'].value_counts()

<ipython-input-179-580dbc3f0fad>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_concat[df_concat['RUNNING_STATUS']=='PULLED'][df_concat['CLASS']=='U']['ESP_FAIL?'].value_counts()


,count
ESP_FAIL?,
Unknown,302
Yes,100
No,25


## Revisando la coherencia entre stop index, stop group y reason for stop or pooh.

In [ ]:
df_concat[df_concat['STOP_GROUP'] == 'ELECTRIC']['STOP_INDEX'].value_counts()

,count
STOP_INDEX,
16.0,1342
14.0,817
18.0,806
17.0,67
15.0,59
21.0,33
20.0,32
19.0,27


In [ ]:
# .uppper a la columna reason for stop or pooh
df_concat['REASON_FOR_STOP_OR_POOH'] = df_concat['REASON_FOR_STOP_OR_POOH'].str.upper()

In [ ]:
df_concat[df_concat['STOP_INDEX'] == 2][['STOP_GROUP', 'REASON_FOR_STOP_OR_POOH']].value_counts()

,,count
STOP_GROUP,REASON_FOR_STOP_OR_POOH,
ELECTIVE,ACIDIZE/STIMULATION,108


In [ ]:
from tabulate import tabulate

# Agrupar por STOP_INDEX, STOP_GROUP y REASON_FOR_STOP_OR_POOH y contar ocurrencias
df_grouped = df_concat.groupby(['STOP_INDEX', 'STOP_GROUP', 'REASON_FOR_STOP_OR_POOH']).size().reset_index(name='COUNT')

# Convertir el DataFrame en una tabla de texto
table_str = tabulate(df_grouped, headers='keys', tablefmt='grid')

# Guardar en un archivo de texto
with open("grouped_stops.txt", "w") as f:
    f.write(table_str)

# Mostrar en pantalla
print("Archivo 'grouped_stops.txt' generado con éxito.")


Archivo 'grouped_stops.txt' generado con éxito.
